# Task 2.4 - Optimizing Hyperparameters - CNN Model

## Table of Contents

#### 1. Importing Libraries and Data
- Cleaned weather observations from 2.2, pleasant weather (predictions)
#### 2. Data Wrangling
#### 3. Reshaping for Modeling
#### 4. Data Split
#### 5. Hyperparameter Optimization
-  Bayesian optimization
#### 6. Running CNN with Optimized Search Parameters
#### 7. Creating Confusion Matrix

## 1. Importing Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)

import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)

2024-09-13 11:54:35.935672: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/giadairene/anaconda3/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [2]:
# Set a path

path = r'/Users/giadairene/Documents/CareerFoundry Data Analytics/Machine Learning with Python/ClimateWins'

In [3]:
# Import cleaned weather observations data

X = pd.read_csv(os.path.join(path, '02 Data', 'Prepared', 'X_cleaned_date.csv'), index_col = False)

In [4]:
# Import predictions data

answers = pd.read_csv(os.path.join(path, '02 Data', 'Original', 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'), index_col = False)

In [5]:
X.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_cloud_cover,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_pressure,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_humidity,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,8,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,1.018,0.20,0.10,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,0.98,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,19600102,1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.00,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,8,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,1.018,0.61,0.30,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,0.62,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,19600103,1,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.00,3.5,3.1,-0.5,6.4,4,0.67,1.017,0.30,0

In [6]:
X.shape

(22950, 137)

In [7]:
answers.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
answers.shape

(22950, 16)

## 2. Data Wrangling

In [9]:
# Drop unnecessary columns

X.drop(['DATE', 'MONTH'], axis=1, inplace=True)

In [10]:
X.shape # observations dataset has the correct shape

(22950, 135)

In [11]:
answers.drop(columns = 'DATE', inplace = True)

In [12]:
answers.shape # predictions dataset has the correct shape

(22950, 15)

## 3. Reshaping for Modeling
- Your X shape will need to be in the form (22950, 15, 9)
- Your y shape will need to be in the form (22950,)
- You can use the argmax function to transform your y set

In [13]:
# Turn X and answers from a df to arrays

X = np.array(X)
y = np.array(answers)

In [14]:
X = X.reshape(-1,15,9)

In [15]:
# Verify shape

X.shape

(22950, 15, 9)

In [16]:
# Use argmax to transform y

y =  np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0])

In [17]:
y.shape

(22950,)

In [18]:
# Check y layout

from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

#### The Bayesian optimization function only accepts y data in “multiclass” and “binary” layouts but not in “multilabel-indicator.”

## 4. Data Split 

In [19]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [20]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


## 5. Bayesian Hyperparameter Optimization

In [21]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # Number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [22]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [23]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), 
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), 
    'epochs':(20, 50),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) 
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/25
15/15 - 4s - 276ms/step - accuracy: 0.6005 - loss: 2.7005
Epoch 2/25
15/15 - 2s - 115ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/25
15/15 - 2s - 109ms/step - accuracy: 0.6440 - loss: 2.6970
Epoch 4/25
15/15 - 2s - 106ms/step - accuracy: 0.6440 - loss: 2.6941
Epoch 5/25
15/15 - 2s - 102ms/step - accuracy: 0.6440 - loss: 2.6916
Epoch 6/25
15/15 - 1s - 98ms/step - accuracy: 0.6440 - loss: 2.6893
Epoch 7/25
15/15 - 1s - 99ms/step - accuracy: 0.6440 - loss: 2.6872
Epoch 8/25
15/15 - 1s - 97ms/step - accuracy: 0.6440 - loss: 2.6852
Epoch 9/25
15/15 - 3s - 193ms/step - accuracy: 0.6440 - loss: 2.6833
Epoch 10/25
15/15 - 2s - 121ms/step - accu

Epoch 18/25
15/15 - 2s - 118ms/step - accuracy: 0.6440 - loss: 2.6685
Epoch 19/25
15/15 - 2s - 117ms/step - accuracy: 0.6440 - loss: 2.6670
Epoch 20/25
15/15 - 2s - 152ms/step - accuracy: 0.6440 - loss: 2.6655
Epoch 21/25
15/15 - 1s - 97ms/step - accuracy: 0.6440 - loss: 2.6639
Epoch 22/25
15/15 - 2s - 102ms/step - accuracy: 0.6440 - loss: 2.6623
Epoch 23/25
15/15 - 2s - 101ms/step - accuracy: 0.6440 - loss: 2.6607
Epoch 24/25
15/15 - 1s - 100ms/step - accuracy: 0.6440 - loss: 2.6591
Epoch 25/25
15/15 - 2s - 119ms/step - accuracy: 0.6440 - loss: 2.6574
4/4 - 0s - 64ms/step
Epoch 1/25
15/15 - 3s - 224ms/step - accuracy: 0.6005 - loss: 2.7045
Epoch 2/25
15/15 - 1s - 99ms/step - accuracy: 0.6439 - loss: 2.7004
Epoch 3/25
15/15 - 1s - 98ms/step - accuracy: 0.6439 - loss: 2.6970
Epoch 4/25
15/15 - 1s - 98ms/step - accuracy: 0.6439 - loss: 2.6942
Epoch 5/25
15/15 - 3s - 169ms/step - accuracy: 0.6439 - loss: 2.6917
Epoch 6/25
15/15 - 1s - 99ms/step - accuracy: 0.6439 - loss: 2.6894
Epoch 7/25

Epoch 28/29
38/38 - 1s - 16ms/step - accuracy: 0.6439 - loss: nan
Epoch 29/29
38/38 - 0s - 8ms/step - accuracy: 0.6439 - loss: nan
10/10 - 0s - 18ms/step
Epoch 1/29
38/38 - 2s - 56ms/step - accuracy: 0.6440 - loss: nan
Epoch 2/29
38/38 - 0s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 3/29
38/38 - 1s - 16ms/step - accuracy: 0.6440 - loss: nan
Epoch 4/29
38/38 - 0s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 5/29
38/38 - 0s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/29
38/38 - 0s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/29
38/38 - 0s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 8/29
38/38 - 0s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 9/29
38/38 - 0s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 10/29
38/38 - 0s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 11/29
38/38 - 0s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 12/29
38/38 - 0s - 9ms/step - accuracy: 0.6440 - loss: nan
Epoch 13/29
38/38 - 0s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 14

Epoch 19/38
17/17 - 2s - 136ms/step - accuracy: 0.8738 - loss: 0.3652
Epoch 20/38
17/17 - 2s - 144ms/step - accuracy: 0.8813 - loss: 0.3481
Epoch 21/38
17/17 - 2s - 126ms/step - accuracy: 0.8792 - loss: 0.3526
Epoch 22/38
17/17 - 2s - 123ms/step - accuracy: 0.8885 - loss: 0.3261
Epoch 23/38
17/17 - 2s - 121ms/step - accuracy: 0.8908 - loss: 0.3183
Epoch 24/38
17/17 - 2s - 122ms/step - accuracy: 0.8869 - loss: 0.3244
Epoch 25/38
17/17 - 2s - 129ms/step - accuracy: 0.8948 - loss: 0.3058
Epoch 26/38
17/17 - 2s - 127ms/step - accuracy: 0.8974 - loss: 0.2975
Epoch 27/38
17/17 - 2s - 125ms/step - accuracy: 0.8985 - loss: 0.2924
Epoch 28/38
17/17 - 3s - 164ms/step - accuracy: 0.9078 - loss: 0.2724
Epoch 29/38
17/17 - 2s - 143ms/step - accuracy: 0.9061 - loss: 0.2728
Epoch 30/38
17/17 - 2s - 127ms/step - accuracy: 0.9083 - loss: 0.2676
Epoch 31/38
17/17 - 2s - 127ms/step - accuracy: 0.9083 - loss: 0.2685
Epoch 32/38
17/17 - 2s - 127ms/step - accuracy: 0.9109 - loss: 0.2620
Epoch 33/38
17/17 - 

Epoch 22/38
17/17 - 2s - 124ms/step - accuracy: 0.8982 - loss: 0.2999
Epoch 23/38
17/17 - 2s - 127ms/step - accuracy: 0.9036 - loss: 0.2788
Epoch 24/38
17/17 - 3s - 152ms/step - accuracy: 0.9033 - loss: 0.2748
Epoch 25/38
17/17 - 3s - 154ms/step - accuracy: 0.9137 - loss: 0.2581
Epoch 26/38
17/17 - 2s - 131ms/step - accuracy: 0.9133 - loss: 0.2547
Epoch 27/38
17/17 - 2s - 130ms/step - accuracy: 0.9187 - loss: 0.2434
Epoch 28/38
17/17 - 2s - 141ms/step - accuracy: 0.9178 - loss: 0.2448
Epoch 29/38
17/17 - 2s - 142ms/step - accuracy: 0.9217 - loss: 0.2326
Epoch 30/38
17/17 - 2s - 129ms/step - accuracy: 0.9230 - loss: 0.2292
Epoch 31/38
17/17 - 3s - 154ms/step - accuracy: 0.9259 - loss: 0.2187
Epoch 32/38
17/17 - 4s - 217ms/step - accuracy: 0.9261 - loss: 0.2163
Epoch 33/38
17/17 - 5s - 273ms/step - accuracy: 0.9293 - loss: 0.2130
Epoch 34/38
17/17 - 3s - 191ms/step - accuracy: 0.9293 - loss: 0.2082
Epoch 35/38
17/17 - 3s - 171ms/step - accuracy: 0.9325 - loss: 0.2000
Epoch 36/38
17/17 - 

...

Epoch 20/27
16/16 - 3s - 168ms/step - accuracy: 0.6439 - loss: nan
Epoch 21/27
16/16 - 1s - 89ms/step - accuracy: 0.6439 - loss: nan
Epoch 22/27
16/16 - 1s - 90ms/step - accuracy: 0.6439 - loss: nan
Epoch 23/27
16/16 - 1s - 90ms/step - accuracy: 0.6439 - loss: nan
Epoch 24/27
16/16 - 1s - 89ms/step - accuracy: 0.6439 - loss: nan
Epoch 25/27
16/16 - 1s - 90ms/step - accuracy: 0.6439 - loss: nan
Epoch 26/27
16/16 - 1s - 91ms/step - accuracy: 0.6439 - loss: nan
Epoch 27/27
16/16 - 3s - 158ms/step - accuracy: 0.6439 - loss: nan
4/4 - 0s - 76ms/step
Epoch 1/27
16/16 - 4s - 254ms/step - accuracy: 0.6440 - loss: nan
Epoch 2/27
16/16 - 2s - 103ms/step - accuracy: 0.6440 - loss: nan
Epoch 3/27
16/16 - 2s - 155ms/step - accuracy: 0.6440 - loss: nan
Epoch 4/27
16/16 - 2s - 106ms/step - accuracy: 0.6440 - loss: nan
Epoch 5/27
16/16 - 1s - 93ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/27
16/16 - 3s - 160ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/27
16/16 - 1s - 92ms/step - accuracy: 0.6440 -

Epoch 19/36
30/30 - 1s - 18ms/step - accuracy: 0.6440 - loss: 1.1599
Epoch 20/36
30/30 - 1s - 22ms/step - accuracy: 0.6440 - loss: 1.1581
Epoch 21/36
30/30 - 1s - 18ms/step - accuracy: 0.6440 - loss: 1.1523
Epoch 22/36
30/30 - 1s - 18ms/step - accuracy: 0.6440 - loss: 1.1354
Epoch 23/36
30/30 - 1s - 21ms/step - accuracy: 0.6440 - loss: 1.1042
Epoch 24/36
30/30 - 1s - 18ms/step - accuracy: 0.6440 - loss: 1.0663
Epoch 25/36
30/30 - 1s - 18ms/step - accuracy: 0.6439 - loss: 1.0437
Epoch 26/36
30/30 - 1s - 23ms/step - accuracy: 0.6433 - loss: 1.0294
Epoch 27/36
30/30 - 1s - 26ms/step - accuracy: 0.6419 - loss: 1.0170
Epoch 28/36
30/30 - 1s - 20ms/step - accuracy: 0.6433 - loss: 1.0072
Epoch 29/36
30/30 - 1s - 18ms/step - accuracy: 0.6423 - loss: 0.9979
Epoch 30/36
30/30 - 1s - 18ms/step - accuracy: 0.6417 - loss: 0.9870
Epoch 31/36
30/30 - 1s - 26ms/step - accuracy: 0.6425 - loss: 0.9764
Epoch 32/36
30/30 - 1s - 17ms/step - accuracy: 0.6433 - loss: 0.9675
Epoch 33/36
30/30 - 1s - 21ms/step

Epoch 30/36
30/30 - 1s - 18ms/step - accuracy: 0.6415 - loss: 1.0177
Epoch 31/36
30/30 - 1s - 20ms/step - accuracy: 0.6390 - loss: 1.0121
Epoch 32/36
30/30 - 1s - 29ms/step - accuracy: 0.6427 - loss: 1.0061
Epoch 33/36
30/30 - 1s - 29ms/step - accuracy: 0.6371 - loss: 1.0034
Epoch 34/36
30/30 - 1s - 33ms/step - accuracy: 0.6426 - loss: 0.9973
Epoch 35/36
30/30 - 1s - 24ms/step - accuracy: 0.6426 - loss: 0.9932
Epoch 36/36
30/30 - 1s - 23ms/step - accuracy: 0.6423 - loss: 0.9888
8/8 - 0s - 30ms/step
| 12        | 0.6289    | 1.079     | 470.1     | 0.9429    | 0.09696   | 35.56     | 2.406     | 1.727     | 2.944     | 0.9628    | 32.66     | 0.4972    | 2.106     |
Epoch 1/22
60/60 - 3s - 54ms/step - accuracy: 0.6478 - loss: 1.1485
Epoch 2/22
60/60 - 2s - 27ms/step - accuracy: 0.7184 - loss: 0.8151
Epoch 3/22
60/60 - 1s - 23ms/step - accuracy: 0.7499 - loss: 0.7223
Epoch 4/22
60/60 - 1s - 23ms/step - accuracy: 0.7707 - loss: 0.6676
Epoch 5/22
60/60 - 3s - 43ms/step - accuracy: 0.7797 -

Epoch 20/22
60/60 - 2s - 33ms/step - accuracy: 0.8666 - loss: 0.3762
Epoch 21/22
60/60 - 2s - 37ms/step - accuracy: 0.8694 - loss: 0.3674
Epoch 22/22
60/60 - 2s - 27ms/step - accuracy: 0.8751 - loss: 0.3553
15/15 - 0s - 24ms/step
| 13        | 0.8241    | 2.564     | 229.5     | 0.6096    | 0.1508    | 21.54     | 1.557     | 2.817     | 1.479     | 0.1534    | 54.05     | 0.9857    | 1.694     |
Epoch 1/31
18/18 - 4s - 219ms/step - accuracy: 0.6248 - loss: 1.1834
Epoch 2/31
18/18 - 2s - 124ms/step - accuracy: 0.7011 - loss: 0.8983
Epoch 3/31
18/18 - 3s - 149ms/step - accuracy: 0.7257 - loss: 0.8065
Epoch 4/31
18/18 - 2s - 138ms/step - accuracy: 0.7418 - loss: 0.7551
Epoch 5/31
18/18 - 2s - 131ms/step - accuracy: 0.7552 - loss: 0.7127
Epoch 6/31
18/18 - 3s - 141ms/step - accuracy: 0.7388 - loss: 0.7302
Epoch 7/31
18/18 - 2s - 122ms/step - accuracy: 0.7563 - loss: 0.6950
Epoch 8/31
18/18 - 2s - 124ms/step - accuracy: 0.7773 - loss: 0.6473
Epoch 9/31
18/18 - 2s - 120ms/step - accuracy: 0

KeyboardInterrupt: 

In [24]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'softsign',
 'batch_size': 460,
 'dropout': 0.7296061783380641,
 'dropout_rate': 0.19126724140656393,
 'epochs': 47,
 'kernel': 1.9444298503238986,
 'layers1': 1,
 'layers2': 2,
 'learning_rate': 0.7631771981307285,
 'neurons': 61,
 'normalization': 0.770967179954561,
 'optimizer': <keras.src.optimizers.adadelta.Adadelta at 0x10bd87790>}

## 6. Running CNN with Optimized Search Parameters

In [25]:
# Set the model with optimized hyperparameters

epochs = 47
batch_size = 460

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

layers1 = 1
layers2 = 2
activation = 'softsign'
kernel = int(round(1.9444298503238986))  # Rounded kernel size for Conv1D
neurons = 61
normalization = 0.770967179954561
dropout = 0.7296061783380641
dropout_rate = 0.19126724140656393
optimizer = Adadelta(learning_rate=0.7631771981307285)  # Instantiate RMSprop with learning rate

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential_67"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_67 (Conv1D)              │ (None, 14, 61)         │         1,159 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 14, 61)         │           244 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_345 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 14, 61)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_346 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_347 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_67 (MaxPooling1D) │ (None, 7, 61)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_67 (Flatten)            │ (None, 427)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_348 (Dense)               │ (None, 15)             │         6,420 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,169 (74.88 KB)

 Trainable params: 19,047 (74.40 KB)

 Non-trainable params: 122 (488.00 B)

In [27]:
# Put the y_test set back into a one-hot configuration

y_train_one_hot = to_categorical(y_train, num_classes=15)

In [28]:
# Check shapes

print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [29]:
# Compile the model with categorical_crossentropy

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [30]:
# Fit the model to the data

model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/47
38/38 - 3s - 90ms/step - accuracy: 0.5807 - loss: 1.4373
Epoch 2/47
38/38 - 3s - 67ms/step - accuracy: 0.6932 - loss: 0.9095
Epoch 3/47
38/38 - 2s - 41ms/step - accuracy: 0.7144 - loss: 0.8258
Epoch 4/47
38/38 - 2s - 46ms/step - accuracy: 0.7406 - loss: 0.7634
Epoch 5/47
38/38 - 2s - 46ms/step - accuracy: 0.7538 - loss: 0.7168
Epoch 6/47
38/38 - 2s - 45ms/step - accuracy: 0.7707 - loss: 0.6781
Epoch 7/47
38/38 - 2s - 48ms/step - accuracy: 0.7804 - loss: 0.6427
Epoch 8/47
38/38 - 2s - 46ms/step - accuracy: 0.7877 - loss: 0.6143
Epoch 9/47
38/38 - 2s - 65ms/step - accuracy: 0.7996 - loss: 0.5851
Epoch 10/47
38/38 - 2s - 48ms/step - accuracy: 0.8074 - loss: 0.5578
Epoch 11/47
38/38 - 2s - 43ms/step - accuracy: 0.8150 - loss: 0.5315
Epoch 12/47
38/38 - 2s - 44ms/step - accuracy: 0.8194 - loss: 0.5166
Epoch 13/47
38/38 - 1s - 39ms/step - accuracy: 0.8259 - loss: 0.5005
Epoch 14/47
38/38 - 2s - 42ms/step - accuracy: 0.8322 - loss: 0.4816
Epoch 15/47
38/38 - 1s - 39ms/step - accura

## 7. Creating Confusion Matrix

In [31]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [32]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [33]:
y_pred = model.predict(X_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [34]:
# Evaluate

print(confusion_matrix(y_test, y_pred, stations))

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3517        93        11       4           3         3       1   
BELGRADE      105       987         0       0           0         0       0   
BUDAPEST       28        36       148       2           0         0       0   
DEBILT         14         6        27      35           0         0       0   
DUSSELDORF      5         2         6       5           5         5       0   
HEATHROW       10         2         4       3           2        60       0   
KASSEL          0         3         1       0           1         1       2   
LJUBLJANA      11         6         4       0           0         4       0   
MAASTRICHT      8         0         0       0           0         0       0   
MADRID         26        19        13       1           2        25       0   
MUNCHENB        5         1         0       0       